In [1]:
import torchvision.models as models
import torch
import torch.nn as nn

from alexnet import load_alexnet
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision

import numpy as np
import matplotlib.pyplot as plt

from hasher import Hasher,set_conv_weights,get_conv_weights
from utils import cifar10_loader,evaluate

In [2]:
class A(nn.Conv2d):
    pass

In [3]:
dir(A)

['__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_get_name',
 '_load_from_state_dict',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'add_module',
 'apply',
 'buffers',
 'children',
 'conv2d_forward',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'half',
 'load_state_dict',
 'modules',
 'named_buffers',
 'named_children',
 'named_modules',
 'named_parameters',
 'parameters',
 'register_backward_hook',
 'register_buffer',


In [10]:
a= A(1,2,3)

In [14]:
a.conv2d_forward()

TypeError: conv2d_forward() missing 2 required positional arguments: 'input' and 'weight'

In [2]:
from scipy.cluster.vq import kmeans

In [3]:
from main import to_hashed

In [4]:
model = load_alexnet().cuda()
train_loader,test_loader = cifar10_loader()

In [5]:
hasher = Hasher(16).cuda()

In [6]:
hasher.load_state_dict(torch.load("./checkpoint/pretrained_hasher_no"))

<All keys matched successfully>

In [7]:
for m in model.modules():
    if isinstance(m,nn.Conv2d) :
        weight = m.weight.cpu().detach().reshape(-1,1)
#         bins,_ = kmeans(weight,16)
        x_pred,prob,bins = hasher(m.weight)
        bins = bins.cpu().detach()
        x = np.abs(weight-bins.reshape(-1))
        x = np.argmin(x,1)
        x = bins[x].reshape(m.weight.size())
#         m.weight = nn.Parameter(torch.from_numpy(x).cuda())
        m.weight = nn.Parameter(x.cuda())

In [8]:
evaluate(model,train_loader,True)

0.73536